In [3]:
import numpy as np
import pandas as pd
import tensorflow as tr
from tensorflow import keras
import os

# VALIDATE DATA
Ensuring files and Sheets have equal values


In [57]:


test_files=[]



file =pd.read_csv("data/test_meta.csv")
#file paths
test_dir="data/test/"
train_dir="data/train/"
#meta record paths
meta_train_dir="data/train_meta.csv"
meta_test_dir="data/test_meta.csv"
#original record paths
org_train_dir=""
org_test_dir=""

def compareFiles(file_path,record_path):
    #prepare sheets
    data = pd.DataFrame(pd.read_csv(record_path))
    # data.head()
    names = data['image_id'] 
    count =0
    # compare png files and records in the Excel sheets
    with os.scandir(file_path)as files:
        for file in files:
            if(str(file.name)!=str(names[count])+".png"):
                print("Inconsistent on line "+str(count)+" for image "+str(file.name)+" and record "+str(names[count])+".png")
            count+=1  
    # Records v files may not be equal
    if(count==len(names)):
        print("Files and folders are equal")
        print(str(count)+" "+str(len(names))) 
    elif(count<len(names)):
        print("File missing")
        print("Files: "+str(count)+" Records: "+str(len(names)))
    elif(count>len(names)):
        print("Record missing")
        print("Files: "+str(count)+" Records: "+str(len(names)))

compareFiles(test_dir,meta_test_dir)
compareFiles(train_dir,meta_train_dir)

Files and folders are equal
3000 3000
Files and folders are equal
15000 15000


# Preprocessing

In [48]:
data.head()
test_dir="data/test/"
train_dir="data/train/"

,image_id,dim0,dim1
0,002a34c58c5b758217ed1f584ccbcfe9,2584,2345
1,004f33259ee4aef671c2b95d54e4be68,3028,2517
2,008bdde2af2462e86fd373a445d0f4cd,2880,2304
3,009bc039326338823ca3aa84381f17f1,2430,1994
4,00a2145de1886cb9eb88869c85d74080,2408,2136
